# Fake DDOS generator

This notebook generates fake DDoS traffic every 0.01 seconds.

In [ ]:
ddos_event = """{"visitor_platform": "mobile","ts_ingest": 1515819844345,"article_title": "Cercanías San Sebastián","visitor_country": "BE","visitor_page_timer": 0,"visitor_os": "ios","article": "https://en.wikipedia.org/wiki/Cercan%C3%ADas_San_Sebasti%C3%A1n","visitor_page_height": 0,"visitor_browser": "unknown"}"""

In [ ]:
# SENDMODE = "kafka"
SENDMODE = "http"

In [ ]:
import json
import time
from datetime import datetime

import requests
from kafka import KafkaProducer
from kafka.errors import KafkaError
from IPython.display import clear_output

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

orig_start = 0
replay_start = time.time() * 1000

counter = 0
i = 0

print("Simulating ddos traffic...")

ddos_event_dict = json.loads(ddos_event)

while True:
    ddos_event_dict['ts_ingest'] = round(time.time() * 1000)
    if SENDMODE == "kafka":        
        future = producer.send('clicks', ddos_event_dict)
        try:
            record_metadata = future.get(timeout=10)
        except KafkaError:
            log.exception()
            pass
    else:
        requests.post("http://localhost:5000/clicks", data=json.dumps(ddos_event_dict).encode("utf-8"), headers={'Content-Type':'application/json'})

    print("✓")
    if i > 1000:
        clear_output()
        i = 0
    i = i+1
    time.sleep(0.001)
